In [ ]:
%load_ext autoreload
%autoreload 2

# Training

In [ ]:
import json
import os
import time
from data.data_set import Dataset
from model.inceptionv3_model import InceptionModel

input_dir = os.environ['IDC_INPUT_DATA_DIR']
output_folder_name = time.strftime("%Y%m%d_%H%M%S") + ('_'+os.environ['GIT_COMMIT'] if 'GIT_COMMIT' in os.environ else '')
output_dir = os.path.join(os.environ['IDC_OUTPUT_DATA_DIR'], output_folder_name)
os.mkdir(output_dir)

dataset_train = Dataset(os.path.join(input_dir, 'csv_train_mutations_notOnlyLuadTiles_small.csv'), num_classes=10)
dataset_valid = Dataset(os.path.join(input_dir, 'csv_valid_mutations_notOnlyLuadTiles_small.csv'), num_classes=10)
model = InceptionModel(num_classes=10, input_shape=(128,128,3), learning_rate=0.01)
print(model)

class_weights = {'norm_cancer': {0:4, 1:1}, 'luad_lusc': {0:1, 1:1}, 'norm_luad_lusc': {0:2, 1:1, 2:1}, 
'mutations':{0:4.0, 1:3.4, 2:5.5, 3:1.0, 4:3.5, 5:2.5, 6:3.0, 7:1.5, 8:3.0, 9:5.0}}
model.train(dataset_train, batch_size=64, epochs=1, output_path=output_dir, validation_dataset=dataset_valid, class_weights=class_weights['mutations'])

# save final model
model.save(output_dir)

# some prediction
data_point = dataset_valid.data_points[0]
prediction = model.make_prediction(data_point)
print('prediction on some sample: %f (reference is %s)' %(prediction, data_point.get_reference_value()))